<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/JSON_in_MySQL_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

https://www.mysqltutorial.org/mysql-json/ <br>
https://www.databasestar.com/mysql-json/

#Install & Test Connectivity

In [ ]:
#!apt-cache policy mysql-server

In [1]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!mysql --version
!/etc/init.d/mysql restart >> /dev/null

mysql  Ver 8.0.32-0ubuntu0.20.04.2 for Linux on x86_64 ((Ubuntu))
su: warning: cannot change directory to /nonexistent: No such file or directory


In [2]:
!mysql -e 'create database praxisDB'
#!mysql -e 'show databases'

#Build Python Client

## Panda Modules

In [3]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb
pd.set_option('display.max_colwidth', None)

In [4]:
DBConn = MySQLdb.connect(db='praxisDB')
#df_mysql = pd.read_sql('select * from emp2;', con=con_mysql)  
df_mysql = pd.read_sql('show databases', con=DBConn)    
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()
#df_mysql

loaded dataframe from MySQL. records: 5


## Pandas Functions

In [5]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    DBConn= MySQLdb.connect(db='praxisDB')
    myCursor = DBConn.cursor()
    try:
        retcode = myCursor.execute(DDL)
        DBConn.commit()
    except :
        print('Error : return code = ',retcode)
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (CMD):
    DBConn= MySQLdb.connect(db='praxisDB')
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

#Create Table

In [6]:
runCMD('drop table if exists product')
ddl = "\
CREATE TABLE product ( \
  id INT, \
  product_name VARCHAR(200), \
  attributes JSON \
);\
"
runCMD(ddl)
runSELECT('desc product')

,Field,Type,Null,Key,Default,Extra
0,id,int,YES,,None,
1,product_name,varchar(200),YES,,None,
2,attributes,json,YES,,None,


#Insert Data - 6 ways

In [7]:
sql = 'INSERT INTO product (id, product_name, attributes) \
VALUES (1, \'Chair\', \'{"color":"brown", "material":"wood", "height":"60cm"}\');'
runCMD(sql)
runSELECT('select * from product')

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"


In [8]:
sql = 'INSERT INTO product (id, product_name, attributes) \
        VALUES (2, "Table", JSON_OBJECT( "color", "brown", "material", "wood", "height", "110cm"));'
runCMD(sql)
runSELECT('select * from product')

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"


In [9]:
sql = 'INSERT INTO product (id, product_name, attributes) \
VALUES (3, \'Desk\',\
  \'{"color":"black", "material":"metal", "drawers":[{"side":"left", "height":"30cm"}, {"side":"left", "height":"40cm"}]}\' \
);'
print(sql)
runCMD(sql)
runSELECT('select * from product')

INSERT INTO product (id, product_name, attributes) VALUES (3, 'Desk',  '{"color":"black", "material":"metal", "drawers":[{"side":"left", "height":"30cm"}, {"side":"left", "height":"40cm"}]}' );


,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"


In [10]:
sql = 'INSERT INTO product (id, product_name, attributes) \
VALUES (4, \'Side Table\', \'{"color":"brown", "material":["metal", "wood"]}\');'
print(sql)
runCMD(sql)
runSELECT('select * from product')

INSERT INTO product (id, product_name, attributes) VALUES (4, 'Side Table', '{"color":"brown", "material":["metal", "wood"]}');


,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"


In [11]:
sql = 'INSERT INTO product (id, product_name, attributes) \
    VALUES (5, \'Dining Table\',  \
    JSON_OBJECT("color", "brown", "material", JSON_ARRAY("wood", "metal") \
    ));'
print(sql)
runCMD(sql)
runSELECT('select * from product')

INSERT INTO product (id, product_name, attributes)     VALUES (5, 'Dining Table',      JSON_OBJECT("color", "brown", "material", JSON_ARRAY("wood", "metal")     ));


,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"


In [12]:
sql = 'INSERT INTO product (id, product_name, attributes) \
        VALUES (6, \'Large Desk\', \
        JSON_OBJECT("color", "white","material", "metal", \
        "drawers", JSON_ARRAY(\'{"side":"left", "height":"50cm"}\', \'{"side":"right", "height":"50cm"}\' ) \
        )\
        );'
print(sql)
runCMD(sql)
runSELECT('select * from product')

INSERT INTO product (id, product_name, attributes)         VALUES (6, 'Large Desk',         JSON_OBJECT("color", "white","material", "metal",         "drawers", JSON_ARRAY('{"side":"left", "height":"50cm"}', '{"side":"right", "height":"50cm"}' )         )        );


,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


#Retrieve Data

In [14]:
runSELECT("SELECT id, product_name, attributes FROM product;")

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [15]:
sql = "\
SELECT id, product_name, \
JSON_EXTRACT(attributes, '$.color') AS color, \
attributes \
FROM product;"
runSELECT(sql)

,id,product_name,color,attributes
0,1,Chair,"""brown""","{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"""brown""","{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"""black""","{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"""brown""","{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,"""brown""","{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"""white""","{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [16]:
sql = "\
SELECT id, product_name, \
JSON_EXTRACT(attributes, '$.height') AS height, \
attributes \
FROM product;"
runSELECT(sql)

,id,product_name,height,attributes
0,1,Chair,"""60cm""","{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"""110cm""","{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,None,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,None,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,None,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,None,"{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [17]:
sql = "\
SELECT id, product_name, \
JSON_EXTRACT(attributes, '$.material') AS material, \
attributes \
FROM product;"
runSELECT(sql)

,id,product_name,material,attributes
0,1,Chair,"""wood""","{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"""wood""","{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"""metal""","{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,"[""metal"", ""wood""]","{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,"[""wood"", ""metal""]","{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"""metal""","{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [19]:
sql = "\
SELECT id, product_name, \
JSON_EXTRACT(attributes, '$.drawers[0]') AS drawers, \
attributes \
FROM product;"
runSELECT(sql)

,id,product_name,drawers,attributes
0,1,Chair,None,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,None,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""side"": ""left"", ""height"": ""30cm""}","{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,None,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,None,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"""{\""side\"":\""left\"", \""height\"":\""50cm\""}""","{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [20]:
sql = "\
SELECT id, product_name, \
JSON_EXTRACT(attributes, '$.drawers[0]') AS drawers0, \
JSON_EXTRACT(attributes, '$.drawers[1]') AS drawers1, \
attributes \
FROM product;"
runSELECT(sql)

,id,product_name,drawers0,drawers1,attributes
0,1,Chair,None,None,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,None,None,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,"{""side"": ""left"", ""height"": ""30cm""}","{""side"": ""left"", ""height"": ""40cm""}","{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,None,None,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,None,None,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,"""{\""side\"":\""left\"", \""height\"":\""50cm\""}""","""{\""side\"":\""right\"", \""height\"":\""50cm\""}""","{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


##Where Condition

In [21]:
sql = "SELECT id, product_name, attributes FROM product \
WHERE JSON_EXTRACT(attributes, '$.color') = 'brown';"
runSELECT(sql)

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
3,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"


In [27]:
sql = "SELECT id, product_name, attributes, \
attributes -> '$.color' as colour \
from product \
WHERE attributes -> '$.color' = 'brown';"
runSELECT(sql)

,id,product_name,attributes,colour
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}","""brown"""
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}","""brown"""
2,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}","""brown"""
3,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}","""brown"""


In [30]:
sql = "SELECT id, product_name, attributes \
from product \
WHERE attributes -> '$.color' = 'brown' \
and attributes -> '$.material' = 'wood';"
runSELECT(sql)

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"


In [31]:
sql = "SELECT id, product_name, attributes \
from product \
WHERE  \
attributes -> '$.material' = 'wood';"
runSELECT(sql)


,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"


In [35]:
sql = 'SELECT id, product_name, \
JSON_CONTAINS( \
  attributes, \
  JSON_OBJECT("material", "wood") \
) AS contain_json, \
attributes \
FROM product;'
runSELECT(sql)


,id,product_name,contain_json,attributes
0,1,Chair,1,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,1,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,3,Desk,0,"{""color"": ""black"", ""drawers"": [{""side"": ""left"", ""height"": ""30cm""}, {""side"": ""left"", ""height"": ""40cm""}], ""material"": ""metal""}"
3,4,Side Table,1,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
4,5,Dining Table,1,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"
5,6,Large Desk,0,"{""color"": ""white"", ""drawers"": [""{\""side\"":\""left\"", \""height\"":\""50cm\""}"", ""{\""side\"":\""right\"", \""height\"":\""50cm\""}""], ""material"": ""metal""}"


In [43]:
sql = 'select id, product_name, attributes from \
(SELECT id, product_name, attributes, \
JSON_CONTAINS(  attributes,   JSON_OBJECT("material", "wood") ) AS contain_json \
FROM product) T \
where \
T.contain_json > 0'
runSELECT(sql)

,id,product_name,attributes
0,1,Chair,"{""color"": ""brown"", ""height"": ""60cm"", ""material"": ""wood""}"
1,2,Table,"{""color"": ""brown"", ""height"": ""110cm"", ""material"": ""wood""}"
2,4,Side Table,"{""color"": ""brown"", ""material"": [""metal"", ""wood""]}"
3,5,Dining Table,"{""color"": ""brown"", ""material"": [""wood"", ""metal""]}"


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)